In [1]:
import pandas as pd
import numpy as np
import re
import scipy
import fancyimpute as fi

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
most_matches = pd.read_csv('most_seasons_unnormalized.csv')
recent_matches = pd.read_csv('recent_seasons_unnormalized.csv')

most_matches = most_matches.fillna(np.nan)
most_matches = most_matches.replace(-1, np.nan)

recent_matches = recent_matches.fillna(np.nan)
recent_matches = recent_matches.replace(-1, np.nan)

most_matches.drop(list(most_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)
recent_matches.drop(list(recent_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)

all_matches = most_matches.append(recent_matches, ignore_index=True)

In [3]:
betting_odds_home = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
betting_odds_draw = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
betting_odds_away = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
betting_odds_all = [betting_odds_home, betting_odds_draw, betting_odds_away]
betting_odds_combined = betting_odds_home + betting_odds_draw + betting_odds_away

rows = all_matches[pd.isnull(all_matches[betting_odds_combined]).any(axis=1)]

In [4]:
for index, row in rows.iterrows():
    if index % 1000 == 0:
        print(index)
    for betting_odds in betting_odds_all:
        mean = np.mean(row[betting_odds])
        all_matches.loc[index, row[betting_odds].index[row[betting_odds].isnull().tolist()]] = mean


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [5]:
home = all_matches.filter(regex='__home_')
away = all_matches.filter(regex='__away_')

home_filled = fi.KNN().fit_transform(home)
home_filled = pd.DataFrame(data=home_filled, columns=home.columns, index=home.index)

away_filled = fi.KNN().fit_transform(away)
away_filled = pd.DataFrame(data=away_filled, columns=away.columns, index=away.index)

Imputing row 1/16576 with 1 missing, elapsed time: 53.860
Imputing row 101/16576 with 1 missing, elapsed time: 53.885
Imputing row 201/16576 with 1 missing, elapsed time: 53.892
Imputing row 301/16576 with 1 missing, elapsed time: 53.899
Imputing row 401/16576 with 1 missing, elapsed time: 53.905
Imputing row 501/16576 with 1 missing, elapsed time: 53.912
Imputing row 601/16576 with 1 missing, elapsed time: 53.919
Imputing row 701/16576 with 0 missing, elapsed time: 53.925
Imputing row 801/16576 with 0 missing, elapsed time: 53.926
Imputing row 901/16576 with 0 missing, elapsed time: 53.926
Imputing row 1001/16576 with 0 missing, elapsed time: 53.926
Imputing row 1101/16576 with 1 missing, elapsed time: 53.932
Imputing row 1201/16576 with 1 missing, elapsed time: 53.942
Imputing row 1301/16576 with 1 missing, elapsed time: 53.951
Imputing row 1401/16576 with 1 missing, elapsed time: 53.960
Imputing row 1501/16576 with 1 missing, elapsed time: 53.967
Imputing row 1601/16576 with 1 missi

Imputing row 15301/16576 with 0 missing, elapsed time: 54.705
Imputing row 15401/16576 with 0 missing, elapsed time: 54.705
Imputing row 15501/16576 with 0 missing, elapsed time: 54.705
Imputing row 15601/16576 with 0 missing, elapsed time: 54.706
Imputing row 15701/16576 with 0 missing, elapsed time: 54.706
Imputing row 15801/16576 with 0 missing, elapsed time: 54.706
Imputing row 15901/16576 with 0 missing, elapsed time: 54.706
Imputing row 16001/16576 with 0 missing, elapsed time: 54.707
Imputing row 16101/16576 with 0 missing, elapsed time: 54.707
Imputing row 16201/16576 with 0 missing, elapsed time: 54.707
Imputing row 16301/16576 with 0 missing, elapsed time: 54.707
Imputing row 16401/16576 with 0 missing, elapsed time: 54.707
Imputing row 16501/16576 with 0 missing, elapsed time: 54.707
Imputing row 1/16576 with 1 missing, elapsed time: 48.259
Imputing row 101/16576 with 1 missing, elapsed time: 48.269
Imputing row 201/16576 with 1 missing, elapsed time: 48.279
Imputing row 301

Imputing row 15301/16576 with 0 missing, elapsed time: 49.297
Imputing row 15401/16576 with 0 missing, elapsed time: 49.298
Imputing row 15501/16576 with 0 missing, elapsed time: 49.298
Imputing row 15601/16576 with 0 missing, elapsed time: 49.298
Imputing row 15701/16576 with 0 missing, elapsed time: 49.298
Imputing row 15801/16576 with 0 missing, elapsed time: 49.298
Imputing row 15901/16576 with 0 missing, elapsed time: 49.299
Imputing row 16001/16576 with 0 missing, elapsed time: 49.299
Imputing row 16101/16576 with 0 missing, elapsed time: 49.299
Imputing row 16201/16576 with 0 missing, elapsed time: 49.299
Imputing row 16301/16576 with 0 missing, elapsed time: 49.299
Imputing row 16401/16576 with 0 missing, elapsed time: 49.300
Imputing row 16501/16576 with 0 missing, elapsed time: 49.300


In [6]:
all_matches_filled = all_matches.copy()
all_matches_filled[home.columns] = home_filled
all_matches_filled[away.columns] = away_filled

In [7]:
cols_to_normalize = ['__home_buildUpPlaySpeed', '__home_buildUpPlayDribbling',
                     '__home_buildUpPlayPassing',
                     '__home_chanceCreationPassing', '__home_chanceCreationCrossing',
                     '__home_chanceCreationShooting', '__home_defencePressure',
                     '__home_defenceAggression', '__home_defenceTeamWidth',
                     '__away_buildUpPlaySpeed', 
                     '__away_buildUpPlayDribbling', '__away_buildUpPlayPassing',
                     '__away_chanceCreationPassing', '__away_chanceCreationCrossing',
                     '__away_chanceCreationShooting', '__away_defencePressure',
                     '__away_defenceAggression', '__away_defenceTeamWidth']

all_matches_filled_normed = all_matches_filled.copy()

# Normalize columns
for column in cols_to_normalize:
    col = np.array(all_matches_filled[column].values.astype(np.float))
    all_matches_filled_normed[column] = (col - col.min()) / (col.max() - col.min())

In [8]:
most_matches_filled = all_matches_filled[:len(most_matches)]
most_matches_filled_normed = all_matches_filled_normed[:len(most_matches)]

recent_matches_filled = all_matches_filled[-len(recent_matches):]
recent_matches_filled_normed = all_matches_filled_normed[-len(recent_matches):]

In [9]:
most_matches_filled.to_csv('most_seasons_filled_unnormalized.csv', index=False)
recent_matches_filled.to_csv('recent_seasons_filled_unnormalized.csv', index=False)

most_matches_filled_normed.to_csv('most_seasons_filled_normalized.csv', index=False)
recent_matches_filled_normed.to_csv('recent_seasons_filled_normalized.csv', index=False)